In [1]:
import talib as talib
import talib as abstract
import pandas as pd
import numpy as np
import copy

In [2]:
data3 = pd.read_csv('3_miniute_future.csv')

In [3]:
data3['datetime']=pd.to_datetime(data3['datetime'])

In [4]:
data3

,datetime,open,high,low,close,volume
0,2020-01-01 09:15:00,32417.3,32466.0,32353.9,32445.0,69920
1,2020-01-01 09:18:00,32445.0,32474.0,32435.0,32465.0,39300
2,2020-01-01 09:21:00,32460.0,32473.1,32445.0,32473.1,21460
3,2020-01-01 09:24:00,32473.0,32490.0,32465.0,32480.1,34780
4,2020-01-01 09:27:00,32480.1,32485.0,32475.0,32485.0,17400
...,...,...,...,...,...,...
62168,2021-12-31 15:15:00,35620.9,35629.3,35616.6,35627.1,32575
62169,2021-12-31 15:18:00,35630.9,35630.9,35605.9,35613.0,41700
62170,2021-12-31 15:21:00,35612.1,35619.1,35604.7,35617.6,26450
62171,2021-12-31 15:24:00,35617.1,35668.2,35616.0,35668.2,59250


In [5]:
import pandas_ta as ta

In [6]:
def atr(data3,n):
    "function to calculate True Range and Average True Range"
    data3 = data3.copy()
    data3['H-L']=abs(data3['high']-data3['low'])
    data3['H-PC']=abs(data3['high']-data3['close'].shift(1))
    data3['L-PC']=abs(data3['low']-data3['close'].shift(1))
    data3['TR']=data3[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    data3['ATR'] = data3['TR'].ewm(com=n,min_periods=n).mean()
    return data3['ATR']

def supertrend(data3,n,m):
    """function to calculate Supertrend given historical candle data
        n = n day ATR - usually 7 day ATR is used
        m = multiplier - usually 2 or 3 is used"""
    data3 = data3.copy()
    data3['ATR'] = atr(data3,n)
    data3["B-U"]=((data3['high']+data3['low'])/2) + m*data3['ATR'] 
    data3["B-L"]=((data3['high']+data3['low'])/2) - m*data3['ATR']
    data3["U-B"]=data3["B-U"]
    data3["L-B"]=data3["B-L"]
    ind = data3.index
    for i in range(n,len(data3)):
        if data3['close'][i-1]<=data3['U-B'][i-1]:
            data3.loc[ind[i],'U-B']=min(data3['B-U'][i],data3['U-B'][i-1])
        else:
            data3.loc[ind[i],'U-B']=data3['B-U'][i]    
    for i in range(n,len(data3)):
        if data3['close'][i-1]>=data3['L-B'][i-1]:
            data3.loc[ind[i],'L-B']=max(data3['B-L'][i],data3['L-B'][i-1])
        else:
            data3.loc[ind[i],'L-B']=data3['B-L'][i]  
    data3['Strend']=np.nan
    for test in range(n,len(data3)):
        if data3['close'][test-1]<=data3['U-B'][test-1] and data3['close'][test]>data3['U-B'][test]:
            data3.loc[ind[test],'Strend']=data3['L-B'][test]
            break
        if data3['close'][test-1]>=data3['L-B'][test-1] and data3['close'][test]<data3['L-B'][test]:
            data3.loc[ind[test],'Strend']=data3['U-B'][test]
            break
    for i in range(test+1,len(data3)):
        if data3['Strend'][i-1]==data3['U-B'][i-1] and data3['close'][i]<=data3['U-B'][i]:
            data3.loc[ind[i],'Strend']=data3['U-B'][i]
        elif  data3['Strend'][i-1]==data3['U-B'][i-1] and data3['close'][i]>=data3['U-B'][i]:
            data3.loc[ind[i],'Strend']=data3['L-B'][i]
        elif data3['Strend'][i-1]==data3['L-B'][i-1] and data3['close'][i]>=data3['L-B'][i]:
            data3.loc[ind[i],'Strend']=data3['L-B'][i]
        elif data3['Strend'][i-1]==data3['L-B'][i-1] and data3['close'][i]<=data3['L-B'][i]:
            data3.loc[ind[i],'Strend']=data3['U-B'][i]
    return data3['Strend']


strend = supertrend(data3,7,3)


In [7]:
strend

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
62168    35689.120667
62169    35689.120667
62170    35682.472943
62171    35682.472943
62172    35682.472943
Name: Strend, Length: 62173, dtype: float64

In [8]:
data3['strend11_2'] = supertrend(data3,11,2)

In [9]:
data3['strend10_1'] = supertrend(data3,10,1)

In [10]:
data3['strend12_3'] = supertrend(data3,12,3)

In [11]:
data3

,datetime,open,high,low,close,volume,strend11_2,strend10_1,strend12_3
0,2020-01-01 09:15:00,32417.3,32466.0,32353.9,32445.0,69920,NaN,NaN,NaN
1,2020-01-01 09:18:00,32445.0,32474.0,32435.0,32465.0,39300,NaN,NaN,NaN
2,2020-01-01 09:21:00,32460.0,32473.1,32445.0,32473.1,21460,NaN,NaN,NaN
3,2020-01-01 09:24:00,32473.0,32490.0,32465.0,32480.1,34780,NaN,NaN,NaN
4,2020-01-01 09:27:00,32480.1,32485.0,32475.0,32485.0,17400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
62168,2021-12-31 15:15:00,35620.9,35629.3,35616.6,35627.1,32575,35649.316504,35640.680714,35679.596455
62169,2021-12-31 15:18:00,35630.9,35630.9,35605.9,35613.0,41700,35649.316504,35640.680714,35679.596455
62170,2021-12-31 15:21:00,35612.1,35619.1,35604.7,35617.6,26450,35649.316504,35636.816913,35679.596455
62171,2021-12-31 15:24:00,35617.1,35668.2,35616.0,35668.2,59250,35587.107317,35614.702807,35679.596455


In [12]:
data3.to_csv('3_miniute_triplesupertrend.csv')

In [13]:
data3.dropna(inplace=True)
#data3.reset_index(inplace=True)

In [14]:
data3

,datetime,open,high,low,close,volume,strend11_2,strend10_1,strend12_3
32,2020-01-01 10:51:00,32297.0,32298.5,32253.3,32265.3,68160,32324.466990,32300.066458,32349.078154
33,2020-01-01 10:54:00,32267.7,32282.5,32260.0,32281.3,16780,32319.501900,32295.258145,32343.957860
34,2020-01-01 10:57:00,32284.0,32284.8,32275.0,32283.0,10920,32319.501900,32295.258145,32343.957860
35,2020-01-01 11:00:00,32283.0,32295.4,32275.0,32293.9,15920,32319.501900,32295.258145,32343.957860
36,2020-01-01 11:03:00,32292.0,32294.2,32280.0,32281.0,8160,32319.501900,32295.258145,32343.957860
...,...,...,...,...,...,...,...,...,...
62168,2021-12-31 15:15:00,35620.9,35629.3,35616.6,35627.1,32575,35649.316504,35640.680714,35679.596455
62169,2021-12-31 15:18:00,35630.9,35630.9,35605.9,35613.0,41700,35649.316504,35640.680714,35679.596455
62170,2021-12-31 15:21:00,35612.1,35619.1,35604.7,35617.6,26450,35649.316504,35636.816913,35679.596455
62171,2021-12-31 15:24:00,35617.1,35668.2,35616.0,35668.2,59250,35587.107317,35614.702807,35679.596455


In [15]:
## to make date time as index

In [16]:
data3=data3.set_index('datetime')

In [17]:
data3

,open,high,low,close,volume,strend11_2,strend10_1,strend12_3
datetime,,,,,,,,
2020-01-01 10:51:00,32297.0,32298.5,32253.3,32265.3,68160,32324.466990,32300.066458,32349.078154
2020-01-01 10:54:00,32267.7,32282.5,32260.0,32281.3,16780,32319.501900,32295.258145,32343.957860
2020-01-01 10:57:00,32284.0,32284.8,32275.0,32283.0,10920,32319.501900,32295.258145,32343.957860
2020-01-01 11:00:00,32283.0,32295.4,32275.0,32293.9,15920,32319.501900,32295.258145,32343.957860
2020-01-01 11:03:00,32292.0,32294.2,32280.0,32281.0,8160,32319.501900,32295.258145,32343.957860
...,...,...,...,...,...,...,...,...
2021-12-31 15:15:00,35620.9,35629.3,35616.6,35627.1,32575,35649.316504,35640.680714,35679.596455
2021-12-31 15:18:00,35630.9,35630.9,35605.9,35613.0,41700,35649.316504,35640.680714,35679.596455
2021-12-31 15:21:00,35612.1,35619.1,35604.7,35617.6,26450,35649.316504,35636.816913,35679.596455


In [19]:
data3.drop(['high','open','low','volume'], axis = 1)

,close,strend11_2,strend10_1,strend12_3
datetime,,,,
2020-01-01 10:51:00,32265.3,32324.466990,32300.066458,32349.078154
2020-01-01 10:54:00,32281.3,32319.501900,32295.258145,32343.957860
2020-01-01 10:57:00,32283.0,32319.501900,32295.258145,32343.957860
2020-01-01 11:00:00,32293.9,32319.501900,32295.258145,32343.957860
2020-01-01 11:03:00,32281.0,32319.501900,32295.258145,32343.957860
...,...,...,...,...
2021-12-31 15:15:00,35627.1,35649.316504,35640.680714,35679.596455
2021-12-31 15:18:00,35613.0,35649.316504,35640.680714,35679.596455
2021-12-31 15:21:00,35617.6,35649.316504,35636.816913,35679.596455


In [20]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62141 entries, 2020-01-01 10:51:00 to 2021-12-31 15:27:00
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   open        62141 non-null  float64
 1   high        62141 non-null  float64
 2   low         62141 non-null  float64
 3   close       62141 non-null  float64
 4   volume      62141 non-null  int64  
 5   strend11_2  62141 non-null  float64
 6   strend10_1  62141 non-null  float64
 7   strend12_3  62141 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 4.3 MB


In [ ]:
import datetime as dt

In [21]:
data3['newcol']=data3.index

In [22]:
data3

,open,high,low,close,volume,strend11_2,strend10_1,strend12_3,newcol
datetime,,,,,,,,,
2020-01-01 10:51:00,32297.0,32298.5,32253.3,32265.3,68160,32324.466990,32300.066458,32349.078154,2020-01-01 10:51:00
2020-01-01 10:54:00,32267.7,32282.5,32260.0,32281.3,16780,32319.501900,32295.258145,32343.957860,2020-01-01 10:54:00
2020-01-01 10:57:00,32284.0,32284.8,32275.0,32283.0,10920,32319.501900,32295.258145,32343.957860,2020-01-01 10:57:00
2020-01-01 11:00:00,32283.0,32295.4,32275.0,32293.9,15920,32319.501900,32295.258145,32343.957860,2020-01-01 11:00:00
2020-01-01 11:03:00,32292.0,32294.2,32280.0,32281.0,8160,32319.501900,32295.258145,32343.957860,2020-01-01 11:03:00
...,...,...,...,...,...,...,...,...,...
2021-12-31 15:15:00,35620.9,35629.3,35616.6,35627.1,32575,35649.316504,35640.680714,35679.596455,2021-12-31 15:15:00
2021-12-31 15:18:00,35630.9,35630.9,35605.9,35613.0,41700,35649.316504,35640.680714,35679.596455,2021-12-31 15:18:00
2021-12-31 15:21:00,35612.1,35619.1,35604.7,35617.6,26450,35649.316504,35636.816913,35679.596455,2021-12-31 15:21:00


In [26]:
(data3.index).time

array([datetime.time(10, 51), datetime.time(10, 54),
       datetime.time(10, 57), ..., datetime.time(15, 21),
       datetime.time(15, 24), datetime.time(15, 27)], dtype=object)